In [1]:
import numpy as np
import cv2

In [2]:
img = cv2.imread(r'img1.jpg')
img.shape

(1280, 954, 3)

In [3]:
width = int(img.shape[1]/2)
height = int(img.shape[0]/2)

In [4]:
img =cv2.resize(img,(width,height))

In [5]:
net = cv2.dnn.readNet('yolov3-tiny.weights','yolov3-tiny.cfg')

In [6]:
classes = []
with open('coco.names.txt','r')as f:
    classes = f.read().splitlines()

In [7]:
# classes

In [8]:
blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB = False, crop = False)

In [9]:
# for each in blob:
#     for n, image in enumerate(each):
#         cv2.imshow(str(n), image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [10]:
net.setInput(blob)
output_layers = net.getUnconnectedOutLayersNames()

In [11]:
print(output_layers)

['yolo_16', 'yolo_23']


In [12]:
layerOutputs = net.forward(output_layers)

In [13]:
print(layerOutputs)

[array([[0.05103783, 0.05888722, 0.14184496, ..., 0.        , 0.        ,
        0.        ],
       [0.04474889, 0.03343038, 0.207488  , ..., 0.        , 0.        ,
        0.        ],
       [0.02270879, 0.03743203, 0.9442982 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.9468365 , 0.9598096 , 0.17143545, ..., 0.        , 0.        ,
        0.        ],
       [0.9520257 , 0.950585  , 0.23067713, ..., 0.        , 0.        ,
        0.        ],
       [0.96725947, 0.95658714, 0.78731525, ..., 0.        , 0.        ,
        0.        ]], dtype=float32), array([[0.02243652, 0.0165489 , 0.01692902, ..., 0.        , 0.        ,
        0.        ],
       [0.02510084, 0.02200937, 0.02862499, ..., 0.        , 0.        ,
        0.        ],
       [0.01882911, 0.02585209, 0.10710219, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97810364, 0.9811948 , 0.02369417, ..., 0.        , 0.        ,
        0.        ],
       [0.98154896

In [15]:
boxes = []
confidences = []
class_ids = []

for each in layerOutputs:
    for detection in each:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > 0.3 :
            center_x = int(detection[0] * width) #1.202
            center_y = int(detection[1] * height) #1.442
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            x = int(center_x - (w/2))
            y = int(center_y - (h/2))
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
print(len(boxes))
            
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4,1)
print(indexes.flatten())

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0,255,size = (len(boxes), 3))

# print(indexes.flatten())

for i in indexes.flatten():
    x,y,w,h = boxes[i]
    label = str(class_ids[i])
    j = int(label)
    print(j)
    confidence = str(round(confidences[i],2))
    color = colors[i]
    cv2.rectangle(img, (x,y), (x+w, y+h),color,3 )
    cv2.putText(img, classes[j]+" "+ confidence+'%', (x,y-20), font, 1,(255,255,255),2)


7
[3 4 5 0 2 1]
0
0
0
0
0
0


In [16]:
cv2.imshow("YOlo Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()